# Задача 3. Реализация контекстного менеджера `safe_write`

In [1]:
"""
Контекстный менеджер для безопасной записи данных в файл.
При возникновении исключения во время записи:
1. Все изменения откатываются, файл возвращается в исходное состояние.
2. Исключение подавляется, и выводится сообщение с указанием типа исключения.
"""
import os

class safe_write:
    def __init__(self, filename: str):
        self.filename = filename
        self.temp_filename = f"{filename}.tmp"

    def __enter__(self):
        # Создаём временный файл для записи
        self.original_exists = os.path.exists(self.filename)
        if self.original_exists:
            # Сохраняем оригинальные данные во временный файл
            with open(self.filename, "r") as original_file, open(self.temp_filename, "w") as temp_file:
                temp_file.write(original_file.read())
        return open(self.filename, "w")

    def __exit__(self, exc_type, exc_value, traceback):
        # Закрываем файл (если он был открыт)
        if exc_type is not None:
            # Возвращаем оригинальное состояние файла
            if self.original_exists:
                with open(self.temp_filename, "r") as temp_file, open(self.filename, "w") as original_file:
                    original_file.write(temp_file.read())
            else:
                os.remove(self.filename)  # Удаляем файл, если его не было до записи
            print(f"Во время записи в файл было возбуждено исключение {exc_type.__name__}")
            return True  # Подавляем исключение
        # Удаляем временный файл, если ошибок не возникло
        if os.path.exists(self.temp_filename):
            os.remove(self.temp_filename)
        return False

# Пример использования контекстного менеджера `safe_write`

In [2]:
# Пример 1: Успешная запись
with safe_write("undertale.txt") as file:
    file.write("Я знаю, что ничего не знаю, но другие не знают и этого.")

with open("undertale.txt") as file:
    print(file.read())  # Ожидаемый вывод: Я знаю, что ничего не знаю, но другие не знают и этого.

Я знаю, что ничего не знаю, но другие не знают и этого.


In [3]:
# Пример 2: Исключение во время записи
try:
    with safe_write("under_tale.txt") as file:
        file.write("Я знаю, что ничего не знаю, но другие не знают и этого.\n")
    with safe_write("under_tale.txt") as file:
        print("Если ты будешь любознательным, то будешь много знающим.", file=file, flush=True)
        raise ValueError  # Принудительно возбуждаем исключение
except Exception as e:
    pass

with open("under_tale.txt") as file:
    print(file.read())  # Ожидаемый вывод:
    # Во время записи в файл было возбуждено исключение ValueError
    # Я знаю, что ничего не знаю, но другие не знают и этого.

Во время записи в файл было возбуждено исключение ValueError

